<a href="https://colab.research.google.com/github/Azimoj/Home-Credit-Default-Risk/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
! pip install jyquickhelper

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [3]:
! pip install shap

In [2]:
# Chargement des librairies
import datetime
import sys
import time
from datetime import datetime
import warnings
import jyquickhelper
import numpy as np
import pandas as pd
import pickle
from joblib import dump, load
from collections import Counter


# Visualisation
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import plotly.graph_objects as go

# Modèle
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
# Standardisation
from sklearn.preprocessing import StandardScaler

# Interprétation
import shap
# Warnings
import warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline
# Chargement à chaud des librairies personnelles
%load_ext autoreload
%autoreload 2
# Valide code pep

In [5]:
# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [109]:
# Original Dataset 
application_train = pd.read_csv('/content/drive/MyDrive/p7/data/application_train.csv')
application_test = pd.read_csv('/content/drive/MyDrive/p7/data/application_test.csv')

In [34]:
application_train.shape, application_test.shape

((307511, 122), (48744, 121))

In [12]:
# dataset after preprocesing

train_set = pd.read_csv('/content/drive/MyDrive/p7/data/data_train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/p7/data/data_test.csv')

In [13]:
test_set.shape

(48744, 896)

In [ ]:
pickle.load(open('/content/drive/MyDrive/p7/best_model.sav', 'rb'))

In [3]:
# Train dataset split
X_train_fin = pd.read_csv('/content/drive/MyDrive/p7/data/X_train_fin.csv')
X_test_fin = pd.read_csv('/content/drive/MyDrive/p7/data/X_test_fin.csv')

In [ ]:
X_train_fin.shape

(246005, 896)

In [82]:
test_set[test_set==np.inf]=np.nan
train_set[train_set==np.inf]=np.nan

In [83]:
test_set= test_set.fillna(0)
train_set= train_set.fillna(0)

In [ ]:
X_test_fin.shape

(61502, 896)

In [5]:
import pickle
best_model = pickle.load(open('/content/drive/MyDrive/p7/best_model.sav', 'rb'))
best_model

LGBMClassifier(boosting_type='goss', class_weight='balanced',
               colsample_bytree=0.78, force_col_wise=True, max_depth=10,
               min_child_samples=54, min_child_weight=42, min_split_gain=0.015,
               num_leaves=35, objective='binary', reg_alpha=0.28,
               reg_lambda=0.03, seed=0, subsample=0.2)

In [6]:
filename =  '/content/drive/MyDrive/p7/best_model.pickle'
outfile = open(filename,'wb')
pickle.dump(best_model, outfile)
outfile.close()

In [ ]:
dump(best_model, '/content/drive/MyDrive/p7/best_model.joblib', compress=1)

['/content/drive/MyDrive/p7/best_model.joblib']

## Scoring (prédictions/Probabilités)

In [9]:
# Copie du jeu de test
test = test_set.copy(deep=True)
# Sauvegarde des identifiants des clients inutiles aux prédictions
id_client = test['SK_ID_CURR']
# Préparation de la matrice à soumettre aux prédictions 
X_test = test.drop('SK_ID_CURR', axis=1)

In [12]:
X_test.shape

(48744, 895)

In [62]:
# Score des prédictions de probabiltés
y_proba = best_model.predict_proba(X_test)[:, 1]

In [63]:
# Score des prédictions de probabiltés
y_proba = best_model.predict_proba(X_test)[:, 1]

In [64]:
# Prédictions pour le client :
# classe 0 : non défaillant, classe 1 : défaillant
y_pred = best_model.predict(X_test)

In [65]:
# Constitution du dataframe pour merge
df_score = pd.DataFrame({'SK_ID_CURR' : id_client,
                         'PRED_CLASSE_CLIENT' : y_pred,
                         'SCORE_CLIENT' : y_proba,
                         'SCORE_CLIENT_%' : np.round(y_proba * 100, 1)})
df_score.head(3)

,SK_ID_CURR,PRED_CLASSE_CLIENT,SCORE_CLIENT,SCORE_CLIENT_%
0,100001,0,0.327993,32.8
1,100005,0,0.356018,35.6
2,100013,0,0.098026,9.8


In [66]:
# Ajout des prédictions au dataframe du jeu de test
df_dashboard = df_score.merge(app_test, on='SK_ID_CURR', how='left')
df_dashboard.head(3)

,SK_ID_CURR,PRED_CLASSE_CLIENT,SCORE_CLIENT,SCORE_CLIENT_%,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_NEW_INSTALLMENTS_PER_LOAN_MIN,CC_NEW_INSTALLMENTS_PER_LOAN_MAX,CC_NEW_INSTALLMENTS_PER_LOAN_MEAN,CC_NEW_INSTALLMENTS_PER_LOAN_SUM,CC_NEW_INSTALLMENTS_PER_LOAN_VAR,CC_COUNT
0,100001,0,0.327993,32.8,0,0,0,0,135000.0,568800.0,...,0.0,0.0,0.0,0.0,27.0,27.0,27.0,810.0,0.0,30.0
1,100005,0,0.356018,35.6,1,0,0,0,99000.0,222768.0,...,0.0,0.0,0.0,0.0,5.0,5.0,5.0,35.0,0.0,7.0
2,100013,0,0.098026,9.8,1,1,0,0,202500.0,663264.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,2075.0,0.0,83.0


In [26]:
df_dashboard.shape

(48744, 899)

In [79]:
# Sauvegarde de df_dashboard
fic_sav_df_dashboard ='/content/drive/MyDrive/p7/df_dashboard_1.pickle'
with open(fic_sav_df_dashboard, 'wb') as f:
    pickle.dump(df_dashboard, f, pickle.HIGHEST_PROTOCOL)

In [40]:
# Chargement du dataframe df_dashboard_1
fic_sav_df_dashboard_1 ='/content/drive/MyDrive/p7/df_dashboard_1.pickle'
with open(fic_sav_df_dashboard_1, 'rb') as df_dashboard_1:
    df_dashboard = pickle.load(df_dashboard_1)
df_dashboard.shape

(48744, 900)

4. **Préparation des données informatives**
4.1. Informations sur le client
Les données informatives sur le client à rendre disponible et lisible pour le chargé de clientèle :

In [67]:
df_infos =application_test[['SK_ID_CURR',
                           'AMT_INCOME_TOTAL', 'NAME_FAMILY_STATUS',
                            'AMT_GOODS_PRICE', 'CNT_CHILDREN',
                            'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
                            'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE']]
df_infos.head(3)

,SK_ID_CURR,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,AMT_GOODS_PRICE,CNT_CHILDREN,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,NAME_INCOME_TYPE
0,100001,135000.0,Married,450000.0,0,Cash loans,Higher education,House / apartment,Working
1,100005,99000.0,Married,180000.0,0,Cash loans,Secondary / secondary special,House / apartment,Working
2,100013,202500.0,Married,630000.0,0,Cash loans,Higher education,House / apartment,Working


In [35]:
df_infos.shape

(48744, 9)

In [68]:
# Ajout des varaibles manquantes au dataframe du dashboard
df_dashboardcc = df_dashboard.merge(df_infos, on='SK_ID_CURR', how='right')
df_dashboardcc.head(3)

,SK_ID_CURR,PRED_CLASSE_CLIENT,SCORE_CLIENT,SCORE_CLIENT_%,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN_x,AMT_INCOME_TOTAL_x,AMT_CREDIT,...,CC_NEW_INSTALLMENTS_PER_LOAN_VAR,CC_COUNT,AMT_INCOME_TOTAL_y,NAME_FAMILY_STATUS,AMT_GOODS_PRICE,CNT_CHILDREN_y,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,NAME_INCOME_TYPE
0,100001,0,0.327993,32.8,0,0,0,0,135000.0,568800.0,...,0.0,30.0,135000.0,Married,450000.0,0,Cash loans,Higher education,House / apartment,Working
1,100005,0,0.356018,35.6,1,0,0,0,99000.0,222768.0,...,0.0,7.0,99000.0,Married,180000.0,0,Cash loans,Secondary / secondary special,House / apartment,Working
2,100013,0,0.098026,9.8,1,1,0,0,202500.0,663264.0,...,0.0,83.0,202500.0,Married,630000.0,0,Cash loans,Higher education,House / apartment,Working


In [69]:
from dataclasses import replace
df_dashboardcc['NEW_AGE']= np.trunc(np.abs(df_dashboardcc['NEW_AGE'])).astype('int8')
df_dashboard.rename(columns={'NEW_AGE': 'YEAR_BIRTH'}, inplace=True)

In [75]:
df_dashboard['DAYS_EMPLOYED'].isnull().sum()

0

In [71]:
df_dashboard.shape

(48744, 899)

In [74]:
df_dashboard[df_dashboard==np.inf]=np.nan

In [76]:
# Transformer DAYS_EMPLOYED en années positives
df_dashboard['YEAR_EMPLOYED'] = \
    np.trunc(np.abs(df_dashboard['DAYS_EMPLOYED'] / 365)).astype('int8')

In [85]:
# Transformer sexe : 0 = Féminin et 1 = Masculin
df_dashboard['SEXE'] = ['Féminin' if row == 0 else 'Masculin'
                        for row in df_dashboard['CODE_GENDER']] 

In [64]:
# Sauvegarde de df_dashboard_2
fic_sav_df_dashboard ='/content/drive/MyDrive/p7/dashboard/df_dashboard_2.pickle'
with open(fic_sav_df_dashboard, 'wb') as f:
    pickle.dump(df_dashboard, f, pickle.HIGHEST_PROTOCOL)

In [65]:
# Chargement du dataframe df_dashboard_2
fic_sav_df_dashboard_2 ='/content/drive/MyDrive/p7/dashboard/df_dashboard_2.pickle'
with open(fic_sav_df_dashboard_2, 'rb') as df_dashboard_2:
    df_dashboard = pickle.load(df_dashboard_2)
df_dashboard.shape

(48744, 900)

**4.2. Variables ayant le plus d'importance sur le modèle**

Les variables ayant le plus d'importance sur le modèle constituent les variables à comparer pour situer notre client par rapport à la moyenne des valeurs de ces variables pour tous les clients et comparer ces valeurs avec celles des 10 plus proches voisins qui ont des valeurs similaires à notre client.
Le but est de tracer un lineplot pour chacune de ces variables déterminantes pour comparer les valeurs du client, les valeurs moyennes sur l'ensemble des clients et les valeurs des 10 plus proches voisins séparés en 2 groupes : défaillant et non-défaillant.
**Variables à comparer ?**

**Features importances**

D'après le notebook sur la modélisation, les 10 variables issus de l'analyse des features importances ayant le plus d'impact sur le meilleur modèle sont :

[X] CREDIT_ANNUITY RATIO

[x] EXT_SOURCE_SUM

[X] INST_PAY_DAYS_PAYMENT_RATIO_MAX

[X] PREV_APP_INTEREST_SHARE_MAX

[X] EXT_SOURCE_1

[X] CREDIT_GOODS_RATIO

[X] CODE GENDER

[X] INST_PAY_AMT_INSTALMENT_SUM

[X] EXT_SOURCE_2

[X] DAYS_BIRTH

[X] AMT_ANNUITY

[X] BUREAU_CURRENT_CREDIT_DEBT_DIFF_MIN

**Shapley values globales**

In [27]:
feature_importances = pickle.load( open('drive/MyDrive/p7/feature_importances.sav', 'rb'))
fi= feature_importances.head(20)
fi.feature

94                 NEW_ANNUITY_CREDIT_RATIO
29                             EXT_SOURCE_3
95                             NEW_EXT_MEAN
28                             EXT_SOURCE_2
27                             EXT_SOURCE_1
12                              OWN_CAR_AGE
6                               AMT_ANNUITY
9                             DAYS_EMPLOYED
8                                DAYS_BIRTH
90                   NEW_DAYS_EMPLOYED_PERC
5                                AMT_CREDIT
96                             NEW_EXT_PROD
97                                  NEW_AGE
11                          DAYS_ID_PUBLISH
0                               CODE_GENDER
10                        DAYS_REGISTRATION
59                   DAYS_LAST_PHONE_CHANGE
117    NAME_EDUCATION_TYPE_Higher education
100           NAME_CONTRACT_TYPE_Cash loans
93                  NEW_ANNUITY_INCOME_PERC
Name: feature, dtype: object

In [32]:
pd.set_option('display.max_rows', None)
 
# storing the dataset as data frame
dataframe = pd.DataFrame(fi.feature, columns = fi.feature)
 
# printing data frame
print(dataframe)

Empty DataFrame
Columns: [NEW_ANNUITY_CREDIT_RATIO, EXT_SOURCE_3, NEW_EXT_MEAN, EXT_SOURCE_2, EXT_SOURCE_1, OWN_CAR_AGE, AMT_ANNUITY, DAYS_EMPLOYED, DAYS_BIRTH, NEW_DAYS_EMPLOYED_PERC, AMT_CREDIT, NEW_EXT_PROD, NEW_AGE, DAYS_ID_PUBLISH, CODE_GENDER, DAYS_REGISTRATION, DAYS_LAST_PHONE_CHANGE, NAME_EDUCATION_TYPE_Higher education, NAME_CONTRACT_TYPE_Cash loans, NEW_ANNUITY_INCOME_PERC]
Index: []


In [33]:
# Liste des variables les plus déterminantes
feat_imp = ['NEW_ANNUITY_CREDIT_RATIO', 
            'EXT_SOURCE_3', 
            'NEW_EXT_MEAN', 
            'EXT_SOURCE_2', 
            'EXT_SOURCE_1', 
            'OWN_CAR_AGE', 
            'AMT_ANNUITY', 
            'DAYS_EMPLOYED', 
            'DAYS_BIRTH', 
            'NEW_DAYS_EMPLOYED_PERC', 
            'AMT_CREDIT', 
            'NEW_EXT_PROD', 
            'NEW_AGE', 
            'DAYS_ID_PUBLISH', 
            'CODE_GENDER', 
            'DAYS_REGISTRATION', 
            'DAYS_LAST_PHONE_CHANGE',
            'NAME_EDUCATION_TYPE_Higher education', 
            'NAME_CONTRACT_TYPE_Cash loans', 
            'NEW_ANNUITY_INCOME_PERC']


In [84]:
# Matrice X - variables les plus déterminantes
X = train_set[feat_imp]

In [85]:
# Standardisation KNN utilise les distances - train set
scaler_std = StandardScaler()
X_std = scaler_std.fit_transform(X)
X_std = pd.DataFrame(X_std, index=X.index, columns=X.columns.to_list())
X_std.shape

(307507, 20)

In [86]:
# Vecteur y
# y = df_dashboard['SCORE_CLIENT'].astype('int64')
y = train_set['TARGET']
y.shape

(307507,)

In [87]:
# Matrice X_test - variables les plus déterminantes
X_test = test_set[feat_imp]

In [88]:
# Standardisation KNN utilise les distances - test set
scaler_std = StandardScaler()
X_test_std = scaler_std.fit_transform(X_test)
X_test_std = pd.DataFrame(X_test_std, index=X_test.index,
                          columns=X_test.columns.to_list())
X_test_std.shape

(48744, 20)

In [89]:
# Vecteur y
y_test = df_dashboard['SCORE_CLIENT'].astype('int64')
y_test.shape

(48744,)

4.2.2. 10 voisins les plus proches du train set

**10 voisins les plus proches du train set**

In [90]:
# Instanciation du modèle
knn = KNeighborsClassifier(10, n_jobs = -1)

In [91]:
# Entraînement du modèle
knn.fit(X_std, y)

KNeighborsClassifier(n_jobs=-1, n_neighbors=10)

In [92]:
# Sauvegarde du modèle entraîné
fic_knn ='/content/drive/MyDrive/p7/dashboard/knn_10_voisins_train.pickle'
with open(fic_knn, 'wb') as f:
    pickle.dump(knn, f, pickle.HIGHEST_PROTOCOL)

In [93]:
# Récupération des index des 10 voisins similaires
liste_index_10_voisins = knn.kneighbors(X_test_std)[1]

In [94]:
# Index des 10 clients voisins dans le train set
df_dashboard['INDEX_10_VOISINS_TRAIN'] = \
    [index for index in liste_index_10_voisins]

In [95]:
# Nombre de clients défaillants parmi les 10 voisins
df_dashboard['NB_10_VOISINS_DEFAILLANT_TRAIN'] = \
    [Counter(train_set['TARGET'].iloc[index])[1]
     for index in liste_index_10_voisins]

In [96]:
# Pourcentage de clients défaillants parmi les 10 voisins
df_dashboard['%_NB_10_VOISINS_DEFAILLANT_TRAIN'] = \
    [(row / 10) * 100
     for row in df_dashboard['NB_10_VOISINS_DEFAILLANT_TRAIN']]

In [97]:
df_dashboard[['SCORE_CLIENT', 'PRED_CLASSE_CLIENT', 
              '%_NB_10_VOISINS_DEFAILLANT_TRAIN']].head(10)

,SCORE_CLIENT,PRED_CLASSE_CLIENT,%_NB_10_VOISINS_DEFAILLANT_TRAIN
0,0.327993,0,10.0
1,0.356018,0,10.0
2,0.098026,0,10.0
3,0.293692,0,0.0
4,0.678324,1,10.0
5,0.361441,0,0.0
6,0.135385,0,0.0
7,0.347489,0,10.0
8,0.126889,0,0.0
9,0.543154,1,10.0


In [98]:
# Sauvegarde de df_dashboard_3
fic_sav_df_dashboard_3 ='/content/drive/MyDrive/p7/dashboard/df_dashboard_3.pickle'
with open(fic_sav_df_dashboard_3, 'wb') as f:
    pickle.dump(df_dashboard, f, pickle.HIGHEST_PROTOCOL)

In [99]:
# Chargement du dataframe df_dashboard_3
fic_sav_df_dashboard_3 ='/content/drive/MyDrive/p7/dashboard/df_dashboard_3.pickle'
with open(fic_sav_df_dashboard_3, 'rb') as df_dashboard_3:
    df_dashboard = pickle.load(df_dashboard_3)
df_dashboard.shape

(48744, 903)

Constitution du dataframe des voisins pour le dashboard

In [100]:
df_index_voisins = \
    pd.DataFrame({'ID_CLIENT' : test_set['SK_ID_CURR'],
                  'INDEX_10_VOISINS' : liste_index_10_voisins.tolist()})
df_index_voisins.shape

(48744, 2)

In [101]:
# Sauvegarde de df_index_voisins
fic_sav_df_index_voisins ='/content/drive/MyDrive/p7/dashboard/df_index_voisins.pickle'
with open(fic_sav_df_index_voisins, 'wb') as f:
    pickle.dump(df_index_voisins, f, pickle.HIGHEST_PROTOCOL)

In [102]:
# Chargement du dataframe df_index_voisins
fic_sav_df_index_voisins ='/content/drive/MyDrive/p7/dashboard/df_index_voisins.pickle'
with open(fic_sav_df_index_voisins, 'rb') as df_index_voisins:
    df_index_voisins = pickle.load(df_index_voisins)
df_index_voisins.shape

(48744, 2)

In [103]:
df_voisin_train = pd.DataFrame({})

In [106]:
# Chargement du dataframe df_voisin_train
fic_sav_df_voisin_train ='/content/drive/MyDrive/p7/dashboard/df_voisin_train.pickle'
with open(fic_sav_df_voisin_train, 'rb') as df_voisin_train:
    df_voisin_train = pickle.load(df_voisin_train)
df_voisin_train.shape

FileNotFoundError: ignored

In [107]:
# ID_CLIENT entre 100001 et 150000 : 6804 clients
mask = df_index_voisins['ID_CLIENT'] >= 100001 
mask2 = df_index_voisins['ID_CLIENT'] < 150000 
df1 = df_index_voisins[mask & mask2]
df1.shape

(6804, 2)

In [131]:
for id_cli, row in zip(df1['ID_CLIENT'],
                       df1['INDEX_10_VOISINS']):
     for index in row:
            var_imp = train_set.iloc[index]
            info = application_train.iloc[index]
            df_voisin_train = df_voisin_train.append(pd.DataFrame({
                                           'ID_CLIENT': [id_cli],
                                           'INDEX_VOISIN': [index],
                                           'AMT_ANNUITY': [var_imp['AMT_ANNUITY']],
                                          # 'BUREAU_CURRENT_CREDIT_DEBT_DIFF_MIN': [var_imp['BUREAU_CURRENT_CREDIT_DEBT_DIFF_MIN']],
                                          # 'BUREAU_CURRENT_CREDIT_DEBT_DIFF_MEAN': [var_imp['BUREAU_CURRENT_CREDIT_DEBT_DIFF_MEAN']],
                                          # 'BUREAU_CURRENT_DEBT_TO_CREDIT_RATIO_MEAN': [var_imp['BUREAU_CURRENT_DEBT_TO_CREDIT_RATIO_MEAN']],
                                          # 'CAR_EMPLOYED_RATIO': [var_imp['CAR_EMPLOYED_RATIO']],
                                           'CODE_GENDER': [var_imp['CODE_GENDER']],
                                           'NEW_ANNUITY_CREDIT_RATIO': [var_imp['NEW_ANNUITY_CREDIT_RATIO']],
                                           'PREV_NEW_CREDIT_GOODS_RATIO_MAX': [var_imp['PREV_NEW_CREDIT_GOODS_RATIO_MAX']],
                                           'DAYS_BIRTH': [var_imp['DAYS_BIRTH']],
                                           'DAYS_ID_PUBLISH': [var_imp['DAYS_ID_PUBLISH']],
                                           'EXT_SOURCE_1': [var_imp['EXT_SOURCE_1']],
                                           'EXT_SOURCE_2': [var_imp['EXT_SOURCE_2']],
                                           'EXT_SOURCE_3': [var_imp['EXT_SOURCE_3']],
                                           'NEW_EXT_MEAN': [var_imp['NEW_EXT_MEAN']],
                                           'NEW_EXT_PROD': [var_imp['NEW_EXT_PROD']],
                                           'FLAG_OWN_CAR': [var_imp['FLAG_OWN_CAR']],
                                           'INSTAL_AMT_INSTALMENT_SUM': [var_imp['INSTAL_AMT_INSTALMENT_SUM']],
                                           'INSTAL_PAYMENT_PERC_MAX': [var_imp['INSTAL_PAYMENT_PERC_MAX']],
                                           'NAME_EDUCATION_TYPE_Higher education': [var_imp['NAME_EDUCATION_TYPE_Higher education']],
                                           'NAME_FAMILY_STATUS_Married ': [var_imp['NAME_FAMILY_STATUS_Married']],
                                           #'POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_SUM': [var_imp['POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_SUM']],
                                           'PREV_APP_INTEREST_SHARE_MAX': [var_imp['PREV_APP_INTEREST_SHARE_MAX']],
                                           'AMT_INCOME_TOTAL': [info['AMT_INCOME_TOTAL']],
                                           'AMT_CREDIT': [info['AMT_CREDIT']],
                                           'NAME_FAMILY_STATUS': [info['NAME_FAMILY_STATUS']],
                                           'DAYS_EMPLOYED': [info['DAYS_EMPLOYED']],
                                           'AMT_GOODS_PRICE': [info['AMT_GOODS_PRICE']],
                                           'CNT_CHILDREN': [info['CNT_CHILDREN']],
                                           'NAME_CONTRACT_TYPE': [info['NAME_CONTRACT_TYPE']],
                                           'NAME_EDUCATION_TYPE': [info['NAME_EDUCATION_TYPE']],
                                           'NAME_HOUSING_TYPE': [info['NAME_HOUSING_TYPE']],
                                           'NAME_INCOME_TYPE': [info['NAME_INCOME_TYPE']]
                                           }), ignore_index=True)         
df_voisin_train.head()

KeyError: ignored

In [128]:
pd.set_option('max_columns', None)
print(train_set.columns.values)

['SK_ID_CURR' 'TARGET' 'CODE_GENDER' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE' 'FLAG_EMAIL'
 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT' 'REGION_RATING_CLIENT_W_CITY'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGINEXPLUATATION_MODE' 'COMMONAREA_MODE'
 'ELEVATORS_MODE' 'ENTRANCES_MODE' '

In [ ]:
# Sauvegarde de df_voisin_train
with open(fic_sav_df_voisin_train, 'wb') as f:
    pickle.dump(df_voisin_train, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
for id_cli, row in zip(df2['ID_CLIENT'],
                       df2['INDEX_10_VOISINS']):
     for index in row:
            var_imp = train_set.iloc[index]
            info = application_train.iloc[index]
            df_voisin_train = df_voisin_train.append(pd.DataFrame({
                                           'ID_CLIENT': [id_cli],
                                           'INDEX_VOISIN': [index],
                                           'AMT_ANNUITY': [var_imp['AMT_ANNUITY']],
                                           'BUREAU_CURRENT_CREDIT_DEBT_DIFF_MIN': [var_imp['BUREAU_CURRENT_CREDIT_DEBT_DIFF_MIN']],
                                           'BUREAU_CURRENT_CREDIT_DEBT_DIFF_MEAN': [var_imp['BUREAU_CURRENT_CREDIT_DEBT_DIFF_MEAN']],
                                           'BUREAU_CURRENT_DEBT_TO_CREDIT_RATIO_MEAN': [var_imp['BUREAU_CURRENT_DEBT_TO_CREDIT_RATIO_MEAN']],
                                           'CAR_EMPLOYED_RATIO': [var_imp['CAR_EMPLOYED_RATIO']],
                                           'CODE_GENDER': [var_imp['CODE_GENDER']],
                                           'CREDIT_ANNUITY_RATIO': [var_imp['CREDIT_ANNUITY_RATIO']],
                                           'CREDIT_GOODS_RATIO': [var_imp['CREDIT_GOODS_RATIO']],
                                           'DAYS_BIRTH': [var_imp['DAYS_BIRTH']],
                                           'DAYS_ID_PUBLISH': [var_imp['DAYS_ID_PUBLISH']],
                                           'EXT_SOURCE_1': [var_imp['EXT_SOURCE_1']],
                                           'EXT_SOURCE_2': [var_imp['EXT_SOURCE_2']],
                                           'EXT_SOURCE_3': [var_imp['EXT_SOURCE_3']],
                                           'EXT_SOURCE_MAX': [var_imp['EXT_SOURCE_MAX']],
                                           'EXT_SOURCE_SUM': [var_imp['EXT_SOURCE_SUM']],
                                           'FLAG_OWN_CAR': [var_imp['FLAG_OWN_CAR']],
                                           'INST_PAY_AMT_INSTALMENT_SUM': [var_imp['INST_PAY_AMT_INSTALMENT_SUM']],
                                           'INST_PAY_DAYS_PAYMENT_RATIO_MAX': [var_imp['INST_PAY_DAYS_PAYMENT_RATIO_MAX']],
                                           'NAME_EDUCATION_TYPE_HIGHER_EDUCATION': [var_imp['NAME_EDUCATION_TYPE_HIGHER_EDUCATION']],
                                           'NAME_FAMILY_STATUS_MARRIED ': [var_imp['NAME_FAMILY_STATUS_MARRIED']],
                                           'POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_SUM': [var_imp['POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_SUM']],
                                           'PREV_APP_INTEREST_SHARE_MAX': [var_imp['PREV_APP_INTEREST_SHARE_MAX']],
                                           'AMT_INCOME_TOTAL': [info['AMT_INCOME_TOTAL']],
                                           'AMT_CREDIT': [info['AMT_CREDIT']],
                                           'NAME_FAMILY_STATUS': [info['NAME_FAMILY_STATUS']],
                                           'DAYS_EMPLOYED': [info['DAYS_EMPLOYED']],
                                           'AMT_GOODS_PRICE': [info['AMT_GOODS_PRICE']],
                                           'CNT_CHILDREN': [info['CNT_CHILDREN']],
                                           'NAME_CONTRACT_TYPE': [info['NAME_CONTRACT_TYPE']],
                                           'NAME_EDUCATION_TYPE': [info['NAME_EDUCATION_TYPE']],
                                           'NAME_HOUSING_TYPE': [info['NAME_HOUSING_TYPE']],
                                           'NAME_INCOME_TYPE': [info['NAME_INCOME_TYPE']]
                                           }), ignore_index=True)         
df_voisin_train.head()